This notebook fixes the issue with our generated fprefcocog. 

We generated from the splitBy=google but we should have used splitBy=umd. Since the google and umd versions are identical except for the train/val/test splits, we can just copy the split value from umd -> google

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
%pip list | grep json

fastjsonschema                    2.18.0
json5                             0.9.14
jsonpointer                       2.4
jsons                             1.6.3
jsonschema                        4.19.1
jsonschema-specifications         2023.7.1
pysimdjson                        5.0.2
python-json-logger                2.0.7
python-lsp-jsonrpc                1.1.1
ujson                             5.8.0
Note: you may need to restart the kernel to use updated packages.


In [101]:
import argparse
import copy
import csv
import decimal
import pickle
import typing
from collections import Counter
from copy import deepcopy
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Set, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import simdjson as json
from IPython.display import display
from pycocotools.coco import COCO
from pycocotools.helpers import CocoClassDistHelper
from tqdm.auto import tqdm

In [105]:
%ls -lah "/home/gbiamby/proj/geo-llm-ret/lib/neg_refer_llm/dataset/refer_seg/refcoco/"
%ls -lah "/home/gbiamby/proj/geo-llm-ret/lib/neg_refer_llm/dataset/refer_seg/fprefcoco_v002/"

total 93M
drwxr-xr-x  2 gbiamby users    5 Sep 22 22:33  ./
drwxr-xr-x 20 gbiamby users   28 Nov  7 22:00  ../
-rw-r--r--  1 gbiamby users 115M Feb  3  2016  instances.json
-rw-r--r--  1 gbiamby users  31M Feb  3  2016 'refs(google).p'
-rw-r--r--  1 gbiamby users  31M Feb  3  2016 'refs(unc).p'
total 126M
drwxr-xr-x  2 gbiamby users    5 Nov  8 18:20  ./
drwxr-xr-x 20 gbiamby users   28 Nov  7 22:00  ../
-rw-r--r--  1 gbiamby users 115M Nov  7 20:31  instances.json
-rw-r--r--  1 gbiamby users  77M Nov  7 20:31 'refs(berkeley).p'
-rw-r--r--  1 gbiamby users  77M Nov  8 18:20 'refs(google_berkeley).p'


In [5]:
REFSEG_DIR = Path("/home/gbiamby/proj/geo-llm-ret/lib/neg_refer_llm/dataset/refer_seg/")
# dataset_json = Path(REFSEG_DIR / "R-refcocog/instances.json")
# coco_dist = CocoClassDistHelper(dataset_json)

In [107]:
VALID_SPLITS = {
    # "R-refcoco": ["unc"],
    # "R-refcoco+": ["unc"],
    # "R-refcocog": ["umd"],
    # "refclef": ["berkeley", "unc"],
    "refcoco": ["unc", "google"],
    "refcoco+": ["unc"],
    "refcocog": [
        "umd",
        "google",
    ],  # ["google", "umd"], # google and umd seem to be identical? THey have the same aggregate stats.
    # "coconegref": ["berkeley"],
    "fprefcoco_v002": ["berkeley"],
    "fprefcoco+_v002": ["berkeley"],
    "fprefcocog_v002": ["berkeley"],
}


def build_refcoco(refseg_path: Path, dataset_name: str, split_by: str = None) -> COCO:
    assert dataset_name in VALID_SPLITS, dataset_name
    if split_by is None:
        split_by = VALID_SPLITS[dataset_name][0]
    else:
        assert split_by in VALID_SPLITS[dataset_name]
    coco = COCO(
        refseg_path / dataset_name / "instances.json",
        is_ref_dataset=True,
        dataset_name=dataset_name,
        split_by=split_by,
    )
    return coco


refg_google = build_refcoco(REFSEG_DIR, "refcoco", "google")
refg_umd = build_refcoco(REFSEG_DIR, "refcoco", "unc")
fprefg_google = build_refcoco(REFSEG_DIR, "fprefcoco_v002", "berkeley")

Loading refs from '/shared/gbiamby/data/refer_seg/refcoco/refs(google).p'
Loaded 50000 refs
loading annotations into memory...
Done (t=1.19s)
creating index...
index created!
Loading refs from '/shared/gbiamby/data/refer_seg/refcoco/refs(unc).p'
Loaded 50000 refs
loading annotations into memory...
Done (t=1.24s)
creating index...
index created!
Loading refs from '/shared/gbiamby/data/refer_seg/fprefcoco_v002/refs(berkeley).p'
Loaded 50000 refs
loading annotations into memory...
Done (t=9.24s)
creating index...
index created!


In [108]:
def compare_refcocos(ref_a: COCO, ref_b: COCO):
    imgs_a = set(ref_a.imgs.keys())
    imgs_b = set(ref_b.imgs.keys())
    print(
        "img_ids same: ",
        imgs_a == imgs_b,
        len(imgs_a),
        len(imgs_b),
        len(imgs_a.intersection(imgs_b)),
    )

    ann_ids_a = set(ref_a.anns.keys())
    ann_ids_b = set(ref_b.anns.keys())
    print(
        "ann_ids same: ",
        ann_ids_a == ann_ids_b,
        len(ann_ids_a),
        len(ann_ids_b),
        len(ann_ids_a.intersection(ann_ids_b)),
    )

    ref_ids_a = set(ref_a.refs.keys())
    ref_ids_b = set(ref_b.refs.keys())
    print(
        "ref_ids same: ",
        ref_ids_a == ref_ids_b,
        len(ref_ids_a),
        len(ref_ids_b),
        len(ref_ids_a.intersection(ref_ids_b)),
    )


compare_refcocos(refg_google, refg_umd)
compare_refcocos(fprefg_google, refg_umd)

img_ids same:  True 19994 19994 19994
ann_ids same:  True 196771 196771 196771
ref_ids same:  True 50000 50000 50000
img_ids same:  True 19994 19994 19994
ann_ids same:  True 196771 196771 196771
ref_ids same:  True 50000 50000 50000


The ref_ids are not consistent between the two datasets, but the actual refering expressions and sentences mapped to them are the same. So we cannot use ref_id as a unique identifier fi we are compare across different ref_seg datasets.

In [41]:
from pycocotools.coco import Ann, Image, Ref

google_img, umd_img = next(
    zip(sorted(refg_google.img_to_refs.items()), sorted(refg_umd.img_to_refs.items()))
)
google_img[0], umd_img[0]
print(
    [(ref["ref_id"], len(ref["sentences"]), ref["sent_ids"]) for ref in google_img[1]]
)
print([(ref["ref_id"], len(ref["sentences"]), ref["sent_ids"]) for ref in umd_img[1]])

sents = []
for ref in google_img[1]:
    for s in ref["sentences"]:
        sents.append(s["sent"])
print(sorted(sents))

print("")
sents = []
for ref in umd_img[1]:
    for s in ref["sentences"]:
        sents.append(s["sent"])
print(sorted(sents))

[(17861, 1, [30065]), (20840, 2, [37026, 37027]), (44673, 2, [92713, 92714])]
[(16534, 1, [30065]), (19208, 2, [37026, 37027]), (40652, 2, [92713, 92714])]
['a pink plastic box which is rectangle', 'a yellow rectangle bowl with broccoli and a brown food in it', 'container holding fruit', 'pink container of tangerines and pineapple', 'the tray that holds the broccoli']

['a pink plastic box which is rectangle', 'a yellow rectangle bowl with broccoli and a brown food in it', 'container holding fruit', 'pink container of tangerines and pineapple', 'the tray that holds the broccoli']


Another result is that refs are not in the same order within an image. ref_ids are not equal, but even if we ignore that, the set of sentences across the image refs are the same, but the ordering of the refs (and therefore the sentences) are not the same between two different datasets.

In [112]:
from collections import defaultdict


def are_refs_equal(ref_a: Ref, ref_b: Ref) -> bool:
    sents_a, sents_b = set(), set()
    [sents_a.add(s["sent"]) for s in ref_a["sentences"]]
    [sents_b.add(s["sent"]) for s in ref_b["sentences"]]
    if sents_a == sents_b:
        return True
    else:
        print("sents_a: ", len(sents_a))
        print("sents_b: ", len(sents_b))
        print("sents_a: ", sents_a)
        print("sents_b: ", sents_b)


def inspect_image(img_id: int, refseg_a: COCO, refseg_b: COCO):
    refs_a = refseg_a.img_to_refs[img_id]
    refs_b = refseg_b.img_to_refs[img_id]
    print("\nrefseg_a: ")
    for ref in refs_a:
        for s in ref["sentences"]:
            print(s["sent"], ref["split"])
    print("\nrefseg_b: ")
    for ref in refs_b:
        for s in ref["sentences"]:
            print(s["sent"], ref["split"])


def copy_splits(refseg_a: COCO, refseg_b: COCO):
    idx = -1
    refseg_b = deepcopy(refseg_b)
    errors = []
    for (img_id_a, refs_a), (img_id_b, refs_b) in tqdm(
        zip(sorted(refseg_a.img_to_refs.items()), sorted(refseg_b.img_to_refs.items())),
        total=len(refseg_b.img_to_refs),
    ):
        idx += 1
        assert len(refs_a) == len(refs_b)
        sents_a, sents_b = {}, {}
        sents_to_ref_a, sents_to_ref_b = defaultdict(list), defaultdict(list)
        for ref_a in refs_a:
            sents_set = set()
            for s in ref_a["sentences"]:
                if ("is_false_premise" not in s) or (not s["is_false_premise"]):
                    sents_a[s["sent"]] = (ref_a, s)
                    sents_set.add(s["sent"])
            sents_to_ref_a[tuple(sents_set)].append(ref_a)
        for ref_b in refs_b:
            sents_set = set()
            for s in ref_b["sentences"]:
                if ("is_false_premise" not in s) or (not s["is_false_premise"]):
                    sents_b[s["sent"]] = (ref_b, s)
                    sents_set.add(s["sent"])
            sents_to_ref_b[tuple(sents_set)].append(ref_b)
        assert len(sents_a) == len(sents_b)
        # Ensure sentences are identical for the image in both datasets:
        assert set(sents_a.keys()) == set(sents_b.keys())
        assert set(sents_to_ref_a.keys()) == set(sents_to_ref_b.keys())
        # assert len(sents_to_ref_a) == len(refs_a)
        if len(sents_to_ref_a) != len(refs_a):
            errors.append(
                {
                    "err": f"len(sents_to_ref_a) != len(refs_a) [{len(sents_to_ref_a)} != {len(refs_a)}]",
                    "sents_to_ref_a": sents_to_ref_a,
                    "sents_to_ref_b": sents_to_ref_b,
                    "refs_a": refs_a,
                    "refs_b": refs_b,
                }
            )
        # assert len(sents_to_ref_b) == len(refs_b)
        if len(sents_to_ref_b) != len(refs_a):
            errors.append(
                {
                    "err": f"len(sents_to_ref_b) != len(refs_b) [{len(sents_to_ref_b)} != {len(refs_b)}]",
                    "sents_to_ref_a": sents_to_ref_a,
                    "sents_to_ref_b": sents_to_ref_b,
                    "refs_a": refs_a,
                    "refs_b": refs_b,
                    "img_id": img_id_a,
                }
            )
        # Now that the refs are ordered the same way, we can loop through them and copy the split data:
        for (_sents_a, _refs_a), (_sents_b, _refs_b) in zip(
            sents_to_ref_a.items(), sents_to_ref_b.items()
        ):
            for _ref_a, _ref_b in zip(_refs_a, _refs_b):
                if idx <= 0:
                    print(_ref_a["split"], ", ", _sents_a)
                    print(_ref_b["split"], ", ", _sents_b)
                _ref_b["split"] = _ref_a["split"]
    return refseg_b, errors


fprefg_google_new, errors = copy_splits(refg_umd, fprefg_google)
print(len(errors))
# inspect_image(529, refg_google, refg_umd)

 56%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                                                                  | 11281/19994 [00:00<00:00, 56461.10it/s]

train ,  ('giraffe on left', 'left giraffe')
train ,  ('giraffe on left', 'left giraffe')
train ,  ('right girafe', 'right giraffe')
val ,  ('right girafe', 'right giraffe')


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19994/19994 [00:00<00:00, 55544.64it/s]

2


In [111]:
display(errors)

[{'err': 'len(sents_to_ref_a) != len(refs_a) [1 != 2]',
  'sents_to_ref_a': {('dog', 'the dog'): {'sent_ids': [76087, 76088, 76089],
    'file_name': 'COCO_train2014_000000274139_0.jpg',
    'ann_id': 9731,
    'ref_id': 26757,
    'image_id': 274139,
    'split': 'train',
    'sentences': [{'tokens': ['the', 'dog'],
      'raw': 'the dog',
      'sent_id': 76087,
      'sent': 'the dog'},
     {'tokens': ['dog'], 'raw': 'dog', 'sent_id': 76088, 'sent': 'dog'},
     {'tokens': ['dog'], 'raw': 'dog', 'sent_id': 76089, 'sent': 'dog'}],
    'category_id': 18}},
  'sents_to_ref_b': {('dog', 'the dog'): {'sent_ids': [76087, 76088, 76089],
    'file_name': 'COCO_train2014_000000274139_0.jpg',
    'ann_id': 9731,
    'ref_id': 26757,
    'image_id': 274139,
    'split': 'train',
    'sentences': [{'tokens': ['the', 'dog'],
      'raw': 'the dog',
      'sent_id': 76087,
      'sent': 'the dog',
      'spcy_WORD': ['the', 'dog'],
      'spcy_DEP': ['det', 'ROOT'],
      'spcy_POS': ['DET', 'NO

In [113]:
def show_train_val_test(refcoco: COCO):
    refs_list = []
    for ref in refcoco.refs.values():
        refs_list.append(
            {
                "ref_id": ref["ref_id"],
                "split": ref["split"],
                "image_id": ref["image_id"],
            }
        )
    df_refs = pd.DataFrame(refs_list)
    display(
        df_refs.groupby("split").agg(
            refs=("ref_id", "nunique"), images=("image_id", "nunique")
        )
    )


show_train_val_test(refg_umd)
show_train_val_test(refg_google)
show_train_val_test(fprefg_google)
show_train_val_test(fprefg_google_new)

,refs,images
split,,
testA,1975,750
testB,1810,750
train,42404,16994
val,3811,1500


,refs,images
split,,
test,5000,4527
train,40000,19213
val,5000,4559


,refs,images
split,,
test,5000,4527
train,40000,19213
val,5000,4559


,refs,images
split,,
testA,1975,750
testB,1810,750
train,42404,16994
val,3811,1500


In [116]:
# now save the new refs:
# refs_path = REFSEG_DIR / "fprefcoco_v002"
# refs_path = Path("/home/gbiamby/proj/geo-llm-ret/output/refcoco_google_enhanced-gb006_remove_guidelines-gpt-3.5-turbo/refer_seg/fprefcoco_v002") # refcocog
refs_path = Path("/home/gbiamby/proj/geo-llm-ret/output/refcoco_google-gb006_remove_guidelines-gpt-3.5-turbo/refer_seg/fprefcoco_v002") # refcoco
assert refs_path.exists()

refs_path = refs_path / "refs(berkeley).p"
print(refs_path)
pickle.dump(fprefg_google_new.refs_data, open(refs_path, "wb"))

/home/gbiamby/proj/geo-llm-ret/output/refcoco_google-gb006_remove_guidelines-gpt-3.5-turbo/refer_seg/fprefcoco_v002/refs(berkeley).p
